In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import platform
import seaborn as sns

## 導入數據集

In [ ]:
theOS = list(platform.uname())[0]
if theOS == 'Windows':
    theOS = '\\'
    theEncode = 'utf-8-sig'
else:
    theOS = '/'
    theEncode = 'utf-8'

df = pd.read_csv('../data/ta_feng_all_months_merged.csv')
df

## 數據整理

In [ ]:
## 時間格式轉換
df['Date'] = pd.to_datetime(df['TRANSACTION_DT'])
## 增加一筆用來計算頻率的欄位
df['count'] = 1
## 創建訂單ID
df['ORDER_ID'] = df.index

## 挑出商品購買頻率前五名的產品種類

In [ ]:
product_five = df.groupby('PRODUCT_SUBCLASS')['count'].sum().sort_values(ascending = False)[:5]
product_five

## 過濾數據集：進留下這五項明星商品種類

In [ ]:
new_df = df[df['PRODUCT_SUBCLASS'].isin(product_five.index)]
new_df

## 購買頻率計算 Frequency

#### 生成每個商品在每個訂單的購買概況

In [ ]:
purchase_list = new_df.pivot_table(index = ['CUSTOMER_ID', 'ORDER_ID', 'PIN_CODE', 'Date', 'AGE_GROUP'],
                                    columns='PRODUCT_SUBCLASS',
                                    aggfunc=sum,
                                    values='AMOUNT'
                                    ).fillna(0).reset_index()
purchase_list

#### 計算每個消費者在一定期間內購買該商品的次數

In [ ]:
purchase_list['frequency'] = 1

frequency = purchase_list.groupby("CUSTOMER_ID",
as_index= False)['frequency'].sum()

del purchase_list['frequency']
frequency

## 最近一次消費計算 Recency

In [ ]:
# 設定今天的日期為最近一位顧客購買的日期,從那天來看過往的銷售狀況
theToday = new_df['Date'].max()
# 找出每個顧客最近一次的購買日期
recent_recency = purchase_list.groupby("CUSTOMER_ID", as_index = False)['Date'].max()
# 計算消費者至今再次購買與上次購買產品的時間差
recent_recency['recency'] =( theToday - recent_recency['Date'] ).astype(str)
recent_recency

In [ ]:
# 将'recency'欄位中的days去除
recent_recency['recency'] = recent_recency['recency'].str.replace('days.*',
                                                                  '',
                                                                  regex = True)
recent_recency

## 合併 Frequency 跟 Recency 

In [ ]:
# 合併recency
recent_recency['recency'] = recent_recency['recency'].astype(int)
purchase_list = recent_recency.merge(purchase_list, on = ['CUSTOMER_ID', 'Date'] ,how='inner')

# 合併 frequency

purchase_list = purchase_list.merge(frequency,
                                    on = ['CUSTOMER_ID'], 
                                    how='inner')
purchase_list

## 檢查 Frequency跟Recency獨有的數值，來決定區間

In [ ]:
print(purchase_list['recency'].unique())
print(purchase_list['frequency'].unique())

## 定義Recency區間

這邊我將近期購買天數劃成六個區間: '0-14天', '16-31天', '32-46天', '47-51天', '52- 66天', '大於66天'

In [ ]:
recency_label =  ['0-14 day', '16-31 day', '32-46 day', '47-51 day', '52- 66day', '>66 day']
recency_cut  = [-1, 14, 31, 46, 51, 66, purchase_list['recency'].max()]
purchase_list['recency_cate'] = pd.cut( 
        purchase_list['recency'],
        recency_cut,
        labels =recency_label)
purchase_list

## 定義Frequency區間

這邊我將每個顧客在四個月期間內購買該商品的數量劃分成六個區間:['0到10個', '11到20個', '21到30個', '31到40個', '41到50個', '大於50個']

In [ ]:
frequency_label =  ['0~10 freq', '11~20 freq', '21~30 freq', '31~40 freq', '41~50 freq', '>50 freq']
frequency_cut  = [0, 10, 20, 30, 40, 50, purchase_list['frequency'].max()]
purchase_list['frequency_cate'] = pd.cut( 
        purchase_list['frequency'] , #目標欄位
        frequency_cut,  #切割條件
        labels =frequency_label) #切割後的分類內容
purchase_list

## RFM 交叉分析

In [ ]:
RF_table = pd.crosstab(purchase_list['frequency_cate'].astype(str),
                       purchase_list['recency_cate'].astype(str))
# 重新排序
RF_table['freq'] = RF_table.index
RF_table = RF_table.sort_values('freq',ascending = False)

collist = ['freq'] + recency_label
RF_table = RF_table[collist]

RF_table

## 定義客群

我將使用Frequency(購買數量)和Recency(近期購買天數)當條件來區分客群

1. Frequency(購買數量)大於40次且Recency(近期購買天數)小於46天的客群: 常貴客
2. Frequency(購買數量)大於40次且Recency(近期購買天數)大於46天的客群: 沉睡客
3. Frequency(購買數量)小於40次且Recency(近期購買天數)大於46天的客群: 流失客
4. 其他則為新顧客

In [ ]:
purchase_list['customer'] = np.where( (purchase_list['frequency'] >=frequency_cut[4]) & (purchase_list['recency']<=recency_cut[3]), '常貴客',
   
                            np.where( (purchase_list['frequency'] >=frequency_cut[4]) & ( purchase_list['recency']>recency_cut[3]), '沉睡客',
                              
                            np.where( (purchase_list['frequency'] < frequency_cut[4]) & ( purchase_list['recency']>recency_cut[3]), '流失客',
                                       
                                       '新顧客'  )))
purchase_list

In [ ]:
## 保存檔案
purchase_list.to_csv('purchase_list.csv', encoding=theEncode)

## 畫出RFM圖

In [ ]:
theOS = list(platform.uname())[0]
if theOS == 'Windows':
    theOS = '\\'
    theEncode = 'utf-8-sig'
else:
    theOS = '/'
    theEncode = 'utf-8'

purchase_list = pd.read_csv('purchase_list.csv')
purchase_list = purchase_list.drop(columns=['Unnamed: 0'])
purchase_list

In [ ]:
## 定義x, y軸標籤
recency_label = ['0-14 day', '16-31 day', '32-46 day', '47-51 day', '52- 66day', '>66 day']
frequency_label = ['0~10 freq', '11~20 freq', '21~30 freq', '31~40 freq', '41~50 freq', '>50 freq']

## 商品種類: 100205 來畫RFM

In [ ]:
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 畫布X軸座標

for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            # 以下為單一小圖表的設定
            sns.barplot(y="100205", # 小圖表Y資料來源欄位
                        data=data, #來源資料表
                        estimator=np.sum,
                        capsize=.2, # 最高點最低點的大小
                        ax=axes[countX, countY]) # 小圖表座標
        ################ 畫X標籤 ################
        if i == '0~10 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 畫Y標籤 ################
        if j == '0-14 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM圖 - 100205', position=(.5,1), fontsize=35) # 設定標題
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20) # 設定X軸標題
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20) # 設定Y軸標題
fig.show()

## 進一步區分顧客群

1.綠色: 常貴客
2.藍色: 新顧客
3.黃色: 沉睡客
4.紅色: 流失客

In [ ]:
target_product = '100205'
# findbig=0
# for i in frequency_label:
#     for j in recency_label:
#         data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
#         if data[target_product].sum() > findbig:
#             findbig = data[target_product].sum()


#--- 繪圖
#先設定畫布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 畫布X軸座標
for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            # 以下為單一小圖表的設定
            sns.barplot(y=target_product, # 小圖表Y資料來源欄位
                        data=data, #來源資料表
                        estimator=np.sum,
                        capsize=.2, # 最高點最低點的大小
                        ax=axes[countX, countY]) # 小圖表座標

        ############### 四個區塊分顏色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #紅色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黃色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #藍色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #綠色
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM圖 - '+target_product, position=(.5,1), fontsize=35) # 設定標題
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20) # 設定X軸標題
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20) # 設定Y軸標題
fig.show()

## 商品種類: 110401 來畫RFM

In [ ]:
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 畫布X軸座標

for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            # 以下為單一小圖表的設定
            sns.barplot(y="110401", # 小圖表Y資料來源欄位
                        data=data, #來源資料表
                        estimator=np.sum,
                        capsize=.2, # 最高點最低點的大小
                        ax=axes[countX, countY]) # 小圖表座標
        ################ 畫X標籤 ################
        if i == '0~10 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 畫Y標籤 ################
        if j == '0-14 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM圖 - 110401', position=(.5,1), fontsize=35) # 設定標題
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20) # 設定X軸標題
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20) # 設定Y軸標題
fig.show()

In [ ]:
target_product = '110401'

#--- 繪圖
#先設定畫布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 畫布X軸座標
for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            # 以下為單一小圖表的設定
            sns.barplot(y=target_product, # 小圖表Y資料來源欄位
                        data=data, #來源資料表
                        estimator=np.sum,
                        capsize=.2, # 最高點最低點的大小
                        ax=axes[countX, countY]) # 小圖表座標

        ############### 四個區塊分顏色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #紅色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黃色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #藍色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #綠色
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM圖 - '+target_product, position=(.5,1), fontsize=35)
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20)
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20)
fig.show()

## 商品種類: 110411 來畫RFM

In [ ]:
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 畫布X軸座標

for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            # 以下為單一小圖表的設定
            sns.barplot(y="110411", # 小圖表Y資料來源欄位
                        data=data, #來源資料表
                        estimator=np.sum,
                        capsize=.2, # 最高點最低點的大小
                        ax=axes[countX, countY]) # 小圖表座標
        ################ 畫X標籤 ################
        if i == '0~10 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 畫Y標籤 ################
        if j == '0-14 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM圖 - 110411', position=(.5,1), fontsize=35) # 設定標題
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20) # 設定X軸標題
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20) # 設定Y軸標題
fig.show()

In [ ]:
target_product = '110411'

#--- 繪圖
#先設定畫布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 畫布X軸座標
for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            # 以下為單一小圖表的設定
            sns.barplot(y=target_product, # 小圖表Y資料來源欄位
                        data=data, #來源資料表
                        estimator=np.sum,
                        capsize=.2, # 最高點最低點的大小
                        ax=axes[countX, countY]) # 小圖表座標

        ############### 四個區塊分顏色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #紅色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黃色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #藍色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #綠色
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM圖 - '+target_product, position=(.5,1), fontsize=35)
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20)
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20)
fig.show()

## 商品種類: 120103 來畫RFM

In [ ]:
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 畫布X軸座標

for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            # 以下為單一小圖表的設定
            sns.barplot(y="120103", # 小圖表Y資料來源欄位
                        data=data, #來源資料表
                        estimator=np.sum,
                        capsize=.2, # 最高點最低點的大小
                        ax=axes[countX, countY]) # 小圖表座標
        ################ 畫X標籤 ################
        if i == '0~10 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 畫Y標籤 ################
        if j == '0-14 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM圖 - 120103', position=(.5,1), fontsize=35) # 設定標題
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20) # 設定X軸標題
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20) # 設定Y軸標題
fig.show()

In [ ]:
target_product = '120103'

#--- 繪圖
#先設定畫布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 畫布X軸座標
for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            # 以下為單一小圖表的設定
            sns.barplot(y=target_product, # 小圖表Y資料來源欄位
                        data=data, #來源資料表
                        estimator=np.sum,
                        capsize=.2, # 最高點最低點的大小
                        ax=axes[countX, countY]) # 小圖表座標

        ############### 四個區塊分顏色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #紅色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黃色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #藍色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #綠色
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM圖 - '+target_product, position=(.5,1), fontsize=35)
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20)
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20)
fig.show()

## 商品種類: 130206 來畫RFM

In [ ]:
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 畫布X軸座標

for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            # 以下為單一小圖表的設定
            sns.barplot(y="130206", # 小圖表Y資料來源欄位
                        data=data, #來源資料表
                        estimator=np.sum,
                        capsize=.2, # 最高點最低點的大小
                        ax=axes[countX, countY]) # 小圖表座標
        ################ 畫X標籤 ################
        if i == '0~10 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 畫Y標籤 ################
        if j == '0-14 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM圖 - 130206', position=(.5,1), fontsize=35) # 設定標題
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20) # 設定X軸標題
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20) # 設定Y軸標題
fig.show()

In [ ]:
target_product = '130206'

#--- 繪圖
#先設定畫布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 畫布X軸座標
for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = purchase_list[(purchase_list['recency_cate']==j) & (purchase_list['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            # 以下為單一小圖表的設定
            sns.barplot(y=target_product, # 小圖表Y資料來源欄位
                        data=data, #來源資料表
                        estimator=np.sum,
                        capsize=.2, # 最高點最低點的大小
                        ax=axes[countX, countY]) # 小圖表座標

        ############### 四個區塊分顏色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #紅色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黃色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #藍色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #綠色
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM圖 - '+target_product, position=(.5,1), fontsize=35)
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20)
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20)
fig.show()

## 綜合 RFM 產品分析 - 留下前三名就好 100205 & 120103 & 110401

In [ ]:
df3 = pd.melt(purchase_list.drop(columns = ['ORDER_ID','Date','recency','frequency', '110411', '130206']), id_vars=['CUSTOMER_ID','customer','recency_cate','frequency_cate','AGE_GROUP', 'PIN_CODE'], var_name='types', value_name='values') 
df3['values'] = pd.to_numeric(df3['values'],errors='coerce')
df3 = df3.dropna()
df3

In [ ]:
#--- 繪圖
#先設定畫布大小
fig, axes = plt.subplots(6, 6,figsize=(25,15))
countX = 0 # 畫布X軸座標
for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = df3[(df3['recency_cate']==j) & (df3['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            # 以下為單一小圖表的設定
            sns.barplot(x="types", # 小圖表X資料來源欄位
                        y="values", # 小圖表Y資料來源欄位
                        data=data, #來源資料表
                        capsize=.2, # 最高點最低點的大小
                        ax=axes[countX, countY]) # 小圖表座標
        # ################ 畫X標籤 ################
        if i == '0~10 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 畫Y標籤 ################
        if j == '0-14 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
            
        ################ 將水、牛奶、香蕉的字變大 ################
        axes[countX][countY].tick_params(labelsize=15)
        ############### 使所有數據的尺碼相同 ################
        axes[countX][countY].set_yticks(range(0,3,1))
        
        
        ############### 四個區塊分顏色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #紅色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黃色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #藍色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #綠色
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM產品面分析圖', position=(.5,1), fontsize=35) # 設定標題
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20) # 設定X軸標題
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20) # 設定Y軸標題
fig.show()

## RFM 市場面分析 - 區域影響 - 以前三名產品為例

In [ ]:
#--- 繪圖
#先設定畫布大小
fig, axes = plt.subplots(6, 6,figsize=(40,25))
countX = 0 # 畫布X軸座標
for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = df3[(df3['recency_cate']==j) & (df3['frequency_cate']==i)]
        if data.shape[0] != 0: # 檢查這個方格有沒有數據
            data = data[['PIN_CODE','types','values']].groupby(['types','PIN_CODE']).sum()
            data =data.groupby(level=1).apply(lambda x:100 * x / float(x.sum())) # 將其數量換成百分比
            data = data.add_suffix('').reset_index() #將三圍度改為二圍度
            data=data.pivot('PIN_CODE', 'types', 'values') # XY軸交換表示
            
            # 以下為單一小圖表的設定
            ax = data.plot.bar(stacked=True, # 設定為堆疊
                              width=0.7,# 柱子的寬度
                              legend = True, 
                              ax =axes[countX, countY] , # 小圖表座標
                              rot=0) #小圖示的標籤旋轉 
            
        ################ 畫X標籤 ################
        if i == '0~10 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 畫Y標籤 ################
        if j == '0-14 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
            
        ################ 將水、牛奶、香蕉的字變大 ################
        axes[countX][countY].tick_params(labelsize=15)
        
        
        ############### 四個區塊分顏色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #紅色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黃色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #藍色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #綠色
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM市場面分析圖', position=(.5,1), fontsize=35) # 設定標題
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20) # 設定X軸標題
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20) # 設定Y軸標題
fig.show()


In [ ]:
## 每個區域對三個產品的消費次數比例
data

In [ ]:
## 將圖例隱藏，剩一個
#--- 繪圖
#先設定畫布大小
fig, axes = plt.subplots(6, 6,figsize=(40,35))
countX = 0 # 畫布X軸座標
for i in frequency_label[::-1]: # 由於axes畫布排列的關係，頻率必須要反著放
    countY = 0 # 畫布Y軸座標
    for j in recency_label: # 近因
        data = df3[(df3['recency_cate']==j) & (df3['frequency_cate']==i)]
        if data.shape[0] != 0: 
            data = data[['PIN_CODE','types','values']].groupby(['types','PIN_CODE']).sum()
            data =data.groupby(level=1).apply(lambda x:100 * x / float(x.sum())) # 將其數量換成百分比
            data = data.add_suffix('').reset_index() #將三圍度改為二圍度
            data=data.pivot('PIN_CODE', 'types', 'values') # XY軸交換表示
            
            # 以下為單一小圖表的設定
            ax = data.plot.bar(stacked=True, # 設定為堆疊
                              width=0.7,# 柱子的寬度
                              legend = False, 
                              ax =axes[countX, countY] , # 小圖表座標
                              rot=0) #小圖示的標籤旋轉
            
        ################ 設定圖例 ################
        if (i == '41~50 freq') and (j == '>66 day'):
            ax.legend(bbox_to_anchor=(1.03, 1), loc=2, fontsize =10) #設定圖例  
            
        ################ 畫X標籤 ################
        if i == '0~10 freq':
            axes[countX][countY].set_xlabel(j, fontsize=17)
            
        ############### 畫Y標籤 ################
        if j == '0-14 day':
            axes[countX][countY].set_ylabel( frequency_label[::-1][countX], fontsize=17)
        else:
            axes[countX][countY].set_ylabel('')
            
            
        ################ 將水、牛奶、香蕉的字變大 ################
        axes[countX][countY].tick_params(labelsize=15)
        
        
        ############### 四個區塊分顏色 ################
        if countY > 2 and countX > 2:
            axes[countX][countY].set(facecolor="#ffcdd2") #紅色
        elif countY > 2 and countX < 3:
            axes[countX][countY].set(facecolor="#FFF9C4") #黃色
        elif countY < 3 and countX > 2:
            axes[countX][countY].set(facecolor="#BBDEFB") #藍色
        else:
            axes[countX][countY].set(facecolor="#B2DFDB") #綠色
            
        countY += 1 
    countX += 1 
fig.suptitle('RFM市場面分析圖', position=(.5,1), fontsize=35) # 設定標題
fig.text(0.5, 0.01, '光顧天數', ha='center', va='center', fontsize=20) # 設定X軸標題
fig.text(0.01, 0.5, '購買頻率', ha='center', va='center', rotation='vertical', fontsize=20) # 設定Y軸標題
fig.show()

## RFM 顧客回購面分析 - 前五名商品

CAI: 顧客活躍度
CAI > 0: 代表活躍
CAI < 0: 代表不活躍

In [ ]:
theOS = list(platform.uname())[0]
if theOS == 'Windows':
    theOS = '\\'
    theEncode = 'utf-8-sig'
else:
    theOS = '/'
    theEncode = 'utf-8'

df = pd.read_csv('../data/ta_feng_all_months_merged.csv')

## 時間格式轉換
df['Date'] = pd.to_datetime(df['TRANSACTION_DT'])
## 增加一筆用來計算頻率的欄位
df['count'] = 1
## 創建訂單ID
df['ORDER_ID'] = df.index

product_five = df.groupby('PRODUCT_SUBCLASS')['count'].sum().sort_values(ascending = False)[:5]
new_df = df[df['PRODUCT_SUBCLASS'].isin(product_five.index)]

purchase_list = new_df.pivot_table(index = ['CUSTOMER_ID', 'ORDER_ID', 'PIN_CODE', 'Date', 'AGE_GROUP'],
                                    columns='PRODUCT_SUBCLASS',
                                    aggfunc=sum,
                                    values='AMOUNT'
                                    ).fillna(0).reset_index()
purchase_list['frequency'] = 1

frequency = purchase_list.groupby("CUSTOMER_ID",
as_index= False)['frequency'].sum()

del purchase_list['frequency']


# 合併 frequency 
purchase_list =purchase_list.merge(frequency, # 即將合併上去的資料表
                                   on = ['CUSTOMER_ID'] # 兩表間連接的key
                                   ,how='inner') # 合併的方式

theToday = purchase_list['Date'].max()
purchase_list['recency'] =( theToday - purchase_list['Date'] ).astype(str)
purchase_list
purchase_list['recency'] = purchase_list['recency'].str.replace('days.*',
                                                                  '',
                                                                  regex = True)
purchase_list['recency'] = purchase_list['recency'].astype(int)
purchase_list

In [ ]:
## 計算顧客購買間隔時間
purchase_list['interval'] = purchase_list.groupby("CUSTOMER_ID", #分類條件
                                  as_index = True # 分類條件是否要取代Index
                                  )['Date'].diff()
#刪除第一次來本店的資料 (因為要算區間，第一筆沒有用，和只來一次的顧客就會被刪掉)
purchase_list.dropna(inplace = True)
# 將時間資料轉成字串
purchase_list['interval'] = purchase_list['interval'].astype(str)
purchase_list

In [ ]:
#將欄位中的days去除
purchase_list['interval'] = purchase_list['interval'].str.replace('days.*', '').astype(int)
purchase_list

In [ ]:
## 計算第幾次來本店消費
purchase_list['cumsum'] = 1
purchase_list['cumsum'] = purchase_list.groupby("CUSTOMER_ID")['cumsum'].cumsum()
purchase_list

In [ ]:
############# 活躍指數 ###########
## 計算平均 平均幾天來一次消費
interval_mean = purchase_list.groupby("CUSTOMER_ID", as_index = False)['interval'].mean()
interval_mean.rename(columns={"interval": "interval_mean"}, inplace = True)
interval_mean

In [ ]:
# 合併平均
purchase_list =purchase_list.merge(interval_mean, # 即將合併上去的資料表
                                   on = ['CUSTOMER_ID'] # 兩表間連接的key
                                   ,how='inner') # 合併的方式
purchase_list

In [ ]:
## 計算顧客活躍度比重
purchase_list['weighted_average'] = purchase_list['interval'] * purchase_list['cumsum'] / (purchase_list['frequency']*(purchase_list['frequency'] -1)/2) / purchase_list['interval_mean']
purchase_list

In [ ]:
## 計算每個顧客的活躍度
clientId_weighted_average = purchase_list.groupby("CUSTOMER_ID", as_index = False)['weighted_average'].sum()
clientId_weighted_average['weighted_average'] = 1-clientId_weighted_average['weighted_average']
clientId_weighted_average

In [ ]:
## 定義顧客屬於活躍還是不活躍
#以0做基準 
clientId_weighted_average['back_probability'] = np.where( clientId_weighted_average['weighted_average'] >= 0, 'good','bad')
clientId_weighted_average

## 重要顧客(活躍)排序

In [ ]:
## 定義級距，有點像PR值，看顧客在所有顧客中的活躍百分比
clientId_weighted_average['percentage'] = (clientId_weighted_average['weighted_average']  - clientId_weighted_average['weighted_average'].min()) / (clientId_weighted_average['weighted_average'].max() - clientId_weighted_average['weighted_average'].min()) *100
clientId_weighted_average = clientId_weighted_average.sort_values(by = 'percentage', ascending = False)
clientId_weighted_average.to_csv('clientId_weighted_average.csv')
clientId_weighted_average

## 商品推薦清單 以前五名商品例

In [ ]:
purchase_list = pd.read_csv('purchase_list.csv', encoding=theEncode)
clientId_weighted_average = pd.read_csv('clientId_weighted_average.csv', encoding=theEncode)

clientId_weighted_average

In [ ]:
########### 創造商品的推薦清單 ###########
recommended_list = purchase_list.groupby("CUSTOMER_ID", #分類條件
                                  as_index = False # 分類條件是否要取代Index
                                  )['100205', '110401', '110411', '120103', '130206'].sum() # 目的欄位 & 計算方式，max, min, mean, sum

recommended_list

sort=[]
for i in range(len(recommended_list)):
    sort.append(' > '.join(recommended_list[['100205', '110401', '110411', '120103', '130206']].iloc[i].sort_values(ascending=False).index.values))

recommended_list['recommended_list'] = sort
recommended_list
# clientId_weighted_average['recommended_list']=sort
# clientId_weighted_average

In [ ]:
clientId_weighted_average = clientId_weighted_average.merge(recommended_list, on = ['CUSTOMER_ID'] ,how='inner')
# clientId_weighted_average.to_csv('clientId_weighted_average2.csv')
clientId_weighted_average = clientId_weighted_average[['CUSTOMER_ID', 'weighted_average', 'back_probability', 'percentage', 'recommended_list']]
clientId_weighted_average

In [ ]:
clientId_weighted_average['recency'] = purchase_list.groupby("CUSTOMER_ID", #分類條件
                                                              as_index = False # 分類條件是否要取代Index
                                                              )['recency'].min()[['recency']] # 目的欄位 & 計算方式，max, min, mean, sum
clientId_weighted_average['frequency'] = purchase_list.groupby("CUSTOMER_ID", #分類條件
                                                              as_index = False # 分類條件是否要取代Index
                                                              )['frequency'].min()['frequency'] # 目的欄位 & 計算方式，max, min, mean, sum
clientId_weighted_average